# L4: Dependency parsing

In this lab, you will implement a simplified version of the dependency parser presented by [Glavaš and Vulić (2021)](http://dx.doi.org/10.18653/v1/2021.eacl-main.270). This parser consists of a transformer encoder followed by a bi-affine layer that computes arc scores for all pairs of words. These scores are then used as logits in a classifier that predicts the position of the head of each word. In contrast to the parser described in the paper, yours will only support unlabelled parsing, that is, you will not implement what the authors call a *relation classifier*. As the encoder, you will use the [uncased DistilBERT base model](https://huggingface.co/docs/transformers/model_doc/distilbert) from the [Transformers](https://huggingface.co/docs/transformers/main/en/index) library, even though every other BERT-based encoder should work equally well.

## Dataset

The data for this lab comes from the English Web Treebank from the [Universal Dependencies Project](http://universaldependencies.org). To read the data, we use the [CoNLL-U Parser](https://pypi.org/project/conllu/) library. The code in the next cell defines a PyTorch [Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) wrapper for the data.

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
!pip install conllu
import conllu

from torch.utils.data import Dataset

class ParserDataset(Dataset):

    def __init__(self, filename):
        super().__init__()
        self.items = []
        with open(filename, 'rt', encoding='utf-8') as fp:
            for tokens in conllu.parse_incr(fp):
                self.items.append([(t['form'], t['head']) for t in tokens if isinstance(t['id'], int)])

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        return self.items[idx]

We can now load the training data:

In [ ]:
TRAIN_DATA = ParserDataset('/content/drive/MyDrive/TDDE09/lab4/en_ewt-ud-train.conllu')

Our data consists of **parsed sentences**. A parsed sentence is represented as a list of pairs. The first component of each pair (a string) represents a word; the second component (an integer) specifies the position of the word’s head, i.e., its parent in the dependency tree. Note that word positions are numbered starting at&nbsp;1; the head position&nbsp;0 represents the root of the tree.

Run the next cell to see an example sentence:

In [ ]:
EXAMPLE_WORDS, EXAMPLE_HEADS = zip(*TRAIN_DATA[531])

EXAMPLE_WORDS, EXAMPLE_HEADS

(('I', 'like', 'yuor', 'blog', '.'), (2, 0, 4, 2, 2))

The example sentence consists of five whitespace-separated words, including the final punctuation mark. The head of the pronoun *I* is the word at position&nbsp;2 – the verb *like*. The head of the word *like* is the root of the tree (position&nbsp;0). The dependents of *like* are *I* (position&nbsp;1), the noun *blog* (position&nbsp;4), and the final punctuation mark. Note that the pronoun *your* (position&nbsp;3) is misspelled as *yuor*.

## Problem 1: Tokenisation (2 points)

To feed parsed sentences to DistilBERT, we need to tokenise them and encode the resulting tokens as integers in the model vocabulary. We start by loading the DistilBERT tokeniser using the [Auto classes](https://huggingface.co/docs/transformers/v4.37.2/en/model_doc/auto):

In [ ]:
from transformers import AutoTokenizer

TOKENIZER = AutoTokenizer.from_pretrained('distilbert-base-uncased')

We can call the tokeniser on the example sentence as shown in the next cell. Note that we use the *is_split_into_words* keyword argument to indicate that the input is already pre-tokenised (split on whitespace).

In [ ]:
EXAMPLE_TOKENS = TOKENIZER(EXAMPLE_WORDS, is_split_into_words=True)
EXAMPLE_TOKENS

{'input_ids': [101, 1045, 2066, 9805, 2953, 9927, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

The output of the tokeniser is an object of class [`BatchEncoding`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding). The code in the following cell shows the list of tokens:

In [ ]:
[TOKENIZER.decode(i) for i in EXAMPLE_TOKENS.input_ids]

['[CLS]', 'i', 'like', 'yu', '##or', 'blog', '.', '[SEP]']

As you can see, the tokeniser adds the special tokens `[CLS]` and `[SEP]` and splits unknown words (here: the misspelled word *yuor*) into sub-word tokens. We will need to keep track of which tokens correspond to which word. To achieve that, we can use the method [`word_to_tokens()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_to_tokens), which gets us the encoded token span (an object of class [`TokenSpan`](https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.TokenSpan)) corresponding to a word in a sequence of the input batch:

In [ ]:
for i, word in enumerate(EXAMPLE_WORDS):
    print(EXAMPLE_TOKENS.word_to_tokens(i), '->', word)

TokenSpan(start=1, end=2) -> I
TokenSpan(start=2, end=3) -> like
TokenSpan(start=3, end=5) -> yuor
TokenSpan(start=5, end=6) -> blog
TokenSpan(start=6, end=7) -> .


Your first task is to code a function `encode()` that takes a tokeniser and a list of sentences and returns the tokeniser&rsquo;s encoded input as well as the corresponding token spans. The following cell contains skeleton code for this function.

In [ ]:
def encode(tokenizer, sentences):
    token_spans = []
    new_sentences = []
    for sentence in sentences:
      zipped_sentence, _ = zip(*sentence)
      new_sentences.append(zipped_sentence)
    tokenized_sentences = tokenizer(new_sentences, is_split_into_words=True, return_tensors='pt', padding=True)
    for i in range(len(new_sentences)):
        sentence_tokenspan = []
        for j, word in enumerate(new_sentences[i]):
            sentence_tokenspan.append(tokenized_sentences.word_to_tokens(i, j))
        token_spans.append(sentence_tokenspan)
    return tokenized_sentences, token_spans

Implement this function to match the following specification:

**encode** (*tokenizer*, *sentences*):

> Uses the specified *tokenizer* to encode a batch of parsed sentences (*sentences*). Returns a pair consisting of a [`BatchEncoding`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding) and a matching batch of token spans (as explained above). The [`BatchEncoding`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding) is the standard batch encoding, but with tensors instead of lists of Python integers. Sentences have been truncated to the maximum acceptable input length.

### 🤞 Test your code

To test you code, call `encode()` on a small number of sentences and check that output matches your expectations.

In [ ]:
tokens, tokenspans = encode(TOKENIZER, [TRAIN_DATA[531], TRAIN_DATA[532]])
print(tokens)
print(tokenspans)

{'input_ids': tensor([[ 101, 1045, 2066, 9805, 2953, 9927, 1012,  102,    0,    0,    0,    0,
            0],
        [ 101, 3531, 4638, 2041, 2026, 2040, 2292, 1996, 3899, 2041, 9927, 1012,
          102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
[[TokenSpan(start=1, end=2), TokenSpan(start=2, end=3), TokenSpan(start=3, end=5), TokenSpan(start=5, end=6), TokenSpan(start=6, end=7)], [TokenSpan(start=1, end=2), TokenSpan(start=2, end=3), TokenSpan(start=3, end=4), TokenSpan(start=4, end=5), TokenSpan(start=5, end=6), TokenSpan(start=6, end=7), TokenSpan(start=7, end=8), TokenSpan(start=8, end=9), TokenSpan(start=9, end=10), TokenSpan(start=10, end=11), TokenSpan(start=11, end=12)]]


## Problem 2: Merging tokens (4 points)

DistilBERT gives us a representation for each *token* in a sentence. To compute scores between pairs of *words*, we will need to combine the token representations that correspond to each word. A standard strategy for this is to take their element-wise mean. The next cell contains skeleton code for a function `merge_tokens()` that implements this strategy.

In [ ]:
def merge_tokens(tokens, token_spans):

    batch_size, num_tokens, hidden_dim = tokens.shape
    max_num_words = max(len(spans) for spans in token_spans)

    result = torch.zeros(batch_size, max_num_words, hidden_dim).to(tokens.device)
    for i in range(batch_size):
      for j,span in enumerate(token_spans[i]):
        result[i,j] = tokens[i, span.start:span.end].mean(dim=0)
    return result





Implement this function to match the following specification:

**merge_tokens** (*tokens*, *token_spans*)

> Takes a batch of token vectors (*tokens*) and a list of matching token spans (*token_spans*) and returns a batch of word-level representations, computed using the element-wise mean. The token vectors are a tensor of shape (*batch_size*, *num_tokens*, *hidden_dim*), where *hidden_dim* is the dimensionality of the DistilBERT representations. The token spans are a nested list containing integer pairs, as computed in Problem&nbsp;1. The result is a tensor of shape (*batch_size*, *max_num_words*, *hidden_dim*), where *max_num_words* denotes the maximum number of words in any sentence in the batch. Entries corresponding to padding are represented by the zero vector of size *hidden_dim*.

### 🤞 Test your code

To test you code, create a sample input to `merge_tokens()` and check that the output matches your expectations.

In [ ]:
from transformers import AutoModel

m = AutoModel.from_pretrained('distilbert-base-uncased')

test = m(**tokens).last_hidden_state

output = merge_tokens(test, tokenspans)
print(output)

tensor([[[ 0.2549,  0.0717,  0.0102,  ..., -0.1623,  0.4906,  0.3904],
         [ 0.4452,  0.2506,  0.5796,  ..., -0.1421,  0.3125, -0.0098],
         [-0.3183,  0.0133,  0.4869,  ..., -0.0500,  0.2308,  0.2228],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.1671,  0.0907,  0.0222,  ...,  0.1022,  0.2065,  0.6775],
         [ 0.4942,  0.1584,  0.4913,  ..., -0.0078,  0.1535, -0.0846],
         [-0.0193,  0.0248,  0.1839,  ..., -0.0406,  0.1255,  0.3385],
         ...,
         [ 0.0492,  0.0931,  0.0876,  ..., -0.1577,  0.2371,  0.4129],
         [ 0.4724,  0.2057,  0.1471,  ...,  0.1096,  0.0147,  0.0115],
         [ 0.4504, -0.1158, -0.4571,  ...,  0.4080, -0.1775, -0.1592]]],
       grad_fn=<CopySlices>)


## Problem 3: Biaffine layer (2 points)

Your next task is to implement the bi-affine layer. Given matrices $X \in \mathbb{R}^{m \times d}$ and $X' \in \mathbb{R}^{n \times d}$, this layer computes a matrix $Y \in \mathbb{R}^{m \times n}$ as

$$
Y = X W X'{}^\top + b
$$

where $W \in \mathbb{R}^{d \times d}$ and $b \in \mathbb{R}$ are learnable weight and bias parameters. In the context of the dependency parser, the matrices $X$ and $X'$ hold the word representations of all dependents and all heads in the input sentence, and the entries of the matrix $Y$ are interpreted as scores of possible dependency arcs. More specifically, the entry $Y_{ij}$ represents the score of an arc from a head word at position&nbsp;$j$ to a dependent at position&nbsp;$i$.


The following cell contains skeleton code for the implementation of the bi-affine layer. Implement this layer according to the specification above.

In [ ]:
import torch.nn as nn

class Biaffine(nn.Module):

    def __init__(self, encoder_dim):
        super().__init__()
        self.W = nn.Parameter(torch.zeros(encoder_dim, encoder_dim))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x1, x2):
        x1W = torch.matmul(x1, self.W)
        Y = torch.matmul(x1W, x2.transpose(-1, -2)) + self.bias

        return Y

**⚠️ Note that your implementation should be able to handle *batches* of input sentences.**

### 🤞 Test your code

To test you code, create a sample input to the bi-affine layer as well as suitable weights and biases and check that the output of the `forward` method matches your expectations

In [ ]:
batch_size = 2
m = 3
n = 4
d = 5

x1 = torch.rand(batch_size, m, d)
x2 = torch.rand(batch_size, n, d)

biaffine_model = Biaffine(d)

output = biaffine_model(x1, x2)

print(output.shape)  # Expected shape: (batch_size, m, n)


torch.Size([2, 3, 4])


## Problem 4: Parser (6 points)

We are now ready to put the two main components of the parser together: the encoder (DistilBert) and the bi-affine layer that computes the arc scores. We also add a dropout layer between the two components.

The following code cell contains skeleton code for the parsing model with the `init()` method already complete. Your task is to implement the `forward()` method. If you are unsure how things should be wired up, have another look at the slides.

In [ ]:
import torch.nn as nn
import torch

from transformers import DistilBertModel, DistilBertPreTrainedModel

class DistilBertForParsing(DistilBertPreTrainedModel):

    def __init__(self, config, dropout=0.1):
        super().__init__(config)
        self.config = config
        self.distilbert = DistilBertModel(config)
        self.dropout = nn.Dropout(dropout)
        self.biaffine = Biaffine(config.hidden_size)

    def forward(self, encoded_input, token_spans):
        distilbert_output = self.distilbert(input_ids=encoded_input.input_ids, attention_mask=encoded_input.attention_mask)
        sequence_output = self.dropout(distilbert_output.last_hidden_state)
        word_embeddings = merge_tokens(sequence_output, token_spans)
        heads = torch.cat((sequence_output[:,0].unsqueeze(1), word_embeddings), dim=1)
        arc_scores = self.biaffine(word_embeddings, heads)

        return arc_scores

Implement the `forward()` method to match the following specification:

**forward** (*encoded_input*, *token_spans*)

> Takes a tokeniser-encoded batch of sentences (*encoded_input*, of type `BatchEncoding`) and a corresponding nested list of token spans (*token_spans*) and returns a tensor with scores for each pair of words. More specifically, the output tensor $Y$ has shape (*batch_size*, *num_words*, *num_words+1*), where the entry $Y_{bij}$ represents the score of an arc from a head word at position&nbsp;$j$ to a dependent at position&nbsp;$i$ in the $b$th sentence of the batch. Note that the number of possible heads is one greater than the number of possible dependents because the possible heads include the root of the dependency tree, which we represent using the special token `[CLS]` (at position&nbsp;0).

### 🤞 Test your code

To test you code, instantiate the parsing model and feed it a small batch of sentences.

In [ ]:
from transformers import DistilBertConfig
config = DistilBertConfig()
model = DistilBertForParsing(config)

tokens, tokenspans = encode(TOKENIZER, [TRAIN_DATA[531]])

scores = model.forward(tokens, tokenspans)

print(scores.shape)



torch.Size([1, 5, 6])


## Data loader

We are now almost ready to train the parser. The missing piece is a data collator that prepares a batch of parsed sentences:

* tokenises the sentences and extracts token spans using `encode()` (Problem&nbsp;1)
* constructs the ground-truth head tensor needed to compute the loss (Problem&nbsp;2)

The code in the next cell implements these two steps. For pseudo-words introduced through padding, we assign a head position of −100. This value is ignored by PyTorch’s cross-entropy loss function.

In [ ]:
import torch

class ParserBatcher(object):

    def __init__(self, tokenizer, device=None):
        self.tokenizer = tokenizer
        self.device = device

    def __call__(self, parser_inputs):
        encoded_input, start_indices = encode(self.tokenizer, parser_inputs)

        # Get the maximal number of words, for padding
        max_num_words = max(len(s) for s in parser_inputs)

        # Construct tensor containing the ground-truth heads
        all_heads = []
        for parser_input in parser_inputs:
            words, heads = zip(*parser_input)
            heads = list(heads)
            heads.extend([-100] * (max_num_words - len(heads)))  # -100 will be ignored
            all_heads.append(heads)
        all_heads = torch.LongTensor(all_heads)

        # Send all data to the specified device
        if self.device:
            encoded_input = encoded_input.to(self.device)
            all_heads = all_heads.to(self.device)

        return encoded_input, start_indices, all_heads

## Training loop

Finally, here is the training loop of the parser. Most of it is quite standard. The training loss of the parser is the cross-entropy between the head scores and the ground truth head positions. In other words, the parser is trained as a classifier that predicts the position of each word&rsquo;s head.

In [ ]:
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#DEVICE = torch.device('mps')

def train(dataset, n_epochs=1, lr=1e-5, batch_size=8):
    # Initialise the tokeniser
    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

    # Initialise the encoder
    model = DistilBertForParsing.from_pretrained('distilbert-base-uncased').to(DEVICE)

    # Initialise the data loader
    data_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=ParserBatcher(tokenizer, device=DEVICE))

    # Initialise the optimiser
    optimizer = Adam(model.parameters(), lr=lr)

    # Train for the specified number of epochs
    for epoch in range(n_epochs):
        model.train()

        # We keep track of the running loss
        running_loss = 0
        n_batches = 0
        with tqdm(total=len(dataset)) as pbar:
            pbar.set_description(f'Epoch {epoch+1}')

            # Process a batch of samples
            for encoded_input, token_spans, gold_heads in data_loader:
                optimizer.zero_grad()
                encoded_input = encoded_input.to(DEVICE)

                # Compute the arc scores
                arc_scores = model.forward(encoded_input, token_spans)
                # shape: [batch_size, num_words, num_words+1]

                # Flatten arc_scores and gold_heads for cross_entropy
                loss = F.cross_entropy(arc_scores.flatten(0, -2), gold_heads.view(-1))
                # shape of the flattened arc_scores: [batch_size * num_words, num_words+1]
                # shape of the flattened gold_heads: [batch_size * num_words]

                # Backward pass
                loss.backward()
                optimizer.step()

                # Update the loss
                running_loss += loss.item()
                n_batches += 1
                pbar.set_postfix(loss=running_loss/n_batches)
                pbar.update(len(token_spans))

    return model

We are now ready to train the parser. With a GPU, you should expect training times of approximately 3&nbsp;minutes per epoch.

In [ ]:
PARSING_MODEL = train(TRAIN_DATA, n_epochs=1)

Some weights of DistilBertForParsing were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['biaffine.bias', 'biaffine.W']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 12544/12544 [02:18<00:00, 90.67it/s, loss=0.727]


## Problem 5: Evaluation (2 points)

Dependency parsers are commonly evaluated using **unlabelled attachment score (UAS)**, which is the percentage of (non-root) words that have been assigned their correct heads. The following cell contains skeleton code for a function `uas()` that computes this score on a given dataset.

In [97]:
def uas(tokenizer, model, filename, batch_size=8):
    data_loader = DataLoader(filename, batch_size=batch_size, collate_fn=ParserBatcher(tokenizer, device=DEVICE))
    model.eval()

    correct_heads = 0
    total_words = 0

    with torch.no_grad():
        for encoded_input, token_spans, gold_heads in data_loader:

            arc_scores = model.forward(encoded_input, token_spans)

            predicted_heads = torch.argmax(arc_scores, dim=-1)
            mask = (gold_heads != -100)
            predicted_heads = predicted_heads[mask]
            gold_heads = gold_heads[mask]
            correct_heads += torch.sum(predicted_heads == gold_heads)
            total_words += torch.sum(mask)

    uas_score = correct_heads.item() / total_words.item() * 100.0
    return uas_score

Implement the `uas()` function to match the following specification:

**uas** (*tokenizer*, *model*, *filename*)

> Takes a tokenizer (*tokenizer*), a trained parsing model (*model*), and the filename of a dataset in the CoNLLU format (*filename*) and returns the unlabelled attachment score of the model on the tokenised dataset.

**⚠️ Note that pseudo-words corresponding to padding must be excluded from the calculation of the UAS.**

The code in the following cell evaluates the trained parser on the development section of the data:

In [98]:
uas(TOKENIZER, PARSING_MODEL, ParserDataset('/content/drive/MyDrive/TDDE09/lab4/en_ewt-ud-dev.conllu'))

88.23412461728101

**Your notebook must contain output demonstrating at least 86% UAS on the development data.**

## Problem 6: Counting trees (6 points)

In the last problem of this lab, we ask you to take a closer look at the graph-theoretic properties of the outputs of your trained model.

The following cell contains code that will use the tokeniser and your trained model to parse the development data and write the gold-standard dependency trees as well as the dependency trees predicted by the parser to a JSON file:

In [99]:
import json

dataset = ParserDataset('/content/drive/MyDrive/TDDE09/lab4/en_ewt-ud-dev.conllu')
data_loader = DataLoader(dataset, collate_fn=ParserBatcher(TOKENIZER, device=DEVICE))

with open('graphs.jsonl', 'wt') as fp:
    for encoded_input, token_spans, gold_heads in data_loader:
        with torch.no_grad():
            head_scores = PARSING_MODEL.forward(encoded_input, token_spans)
            pred_heads = torch.argmax(head_scores, dim=-1)
        for gold, pred in zip(gold_heads, pred_heads):
            mask = gold.ne(-100)
            print(json.dumps((gold[mask].tolist(), pred[mask].tolist())), file=fp)

In [ ]:
!python /content/drive/MyDrive/TDDE09/lab4/analyze.py graphs.jsonl

{'total': 2001, 'gold': {'acyclic': 2001, 'connected': 2001, 'tree': 2001, 'projective': 1959}, 'pred': {'acyclic': 2001, 'connected': 1833, 'tree': 1833, 'projective': 1596}}


The lab directory contains a script `analyze.py` that you can use to compute statistics on various graph-theoretic properties: total number of graphs, number of acyclic graphs, connected graphs, trees, and projective trees. Document your exploration in a short reflection piece (ca.&nbsp;150 words). Respond to the following prompts

* What are the results of this exploration? Summarise the statistics you obtained.
* Based on what you know about the parser, did you expect your results? Was there anything surprising in them?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

These are the obtained statistics: {'total': 2001, 'gold': {'acyclic': 2001, 'connected': 2001, 'tree': 2001, 'projective': 1959},
'pred': {'acyclic': 2001, 'connected': 1833, 'tree': 1833, 'projective': 1596}}

These results show that the predictions were very good. The predictions correctly identified all acyclic graphs meaning that it was good at avoiding cycles. The number of connected graphs were 1833 of 2001 which is a high percent but not perfect. Finally the predictions were worse at identifying projected trees. I found it interesting that the parser were as good as it was. But that it was better at identifing connected and acyclic graphs than projective trees seems reasonable with the parsers objective (understand how words relate to eachother in a sentence) in mind. Since the parser does not focus directly on creating/identifing projective trees it should not do it perfectly. I learned this by combining knowledge from the lecture slides and the lab experience.

**🥳 Congratulations on finishing this lab! 🥳**